In [13]:
import pandas as pd

# Specify the file name
file_name = 'autoscout24-germany-dataset.csv'

# Read the JSON file into a DataFrame
df = pd.read_csv(file_name)

# Display the DataFrame
df.head()


,mileage,make,model,fuel,gear,offerType,price,hp,year
0,235000,BMW,316,Diesel,Manual,Used,6800,116.0,2011
1,92800,Volkswagen,Golf,Gasoline,Manual,Used,6877,122.0,2011
2,149300,SEAT,Exeo,Gasoline,Manual,Used,6900,160.0,2011
3,96200,Renault,Megane,Gasoline,Manual,Used,6950,110.0,2011
4,156000,Peugeot,308,Gasoline,Manual,Used,6950,156.0,2011


In [3]:
df.size

417645

In [14]:
# Check data types of each element in the DataFrame
dtypes_df = df.applymap(type)

# Print columns with lists
columns_with_lists = dtypes_df.columns[dtypes_df.applymap(lambda x: 'list' in str(x)).any()]
print("Columns with lists:", columns_with_lists)

C:\Users\User\AppData\Local\Temp\ipykernel_13140\52072551.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dtypes_df = df.applymap(type)
C:\Users\User\AppData\Local\Temp\ipykernel_13140\52072551.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  columns_with_lists = dtypes_df.columns[dtypes_df.applymap(lambda x: 'list' in str(x)).any()]


Columns with lists: Index([], dtype='object')


In [15]:
# Convert columns with lists to strings
df[columns_with_lists] = df[columns_with_lists].astype(str)

In [16]:
# Drop duplicate
df = df.drop_duplicates()

In [8]:
# Print summary of null values in each column
print(df.isnull().sum())


mileage        0
make           0
model        137
fuel           0
gear         175
offerType      0
price          0
hp            24
year           0
dtype: int64


In [9]:
df.dtypes

mileage        int64
make          object
model         object
fuel          object
gear          object
offerType     object
price          int64
hp           float64
year           int64
dtype: object

In [20]:

# Check if data type is the same for all values within each column
are_all_columns_same_dtype = all(df[col].apply(type).nunique() == 1 for col in df.columns)

if are_all_columns_same_dtype:
    print("Data type is the same for all values within each column.")
else:
    print("Data type varies within some columns.")

Data type is the same for all values within each column.


In [19]:
import pandas as pd

# Assuming df is your DataFrame
# Create a sample DataFrame for demonstration
# df = pd.DataFrame({...})

# Check if data type is the same for all values within each column
are_all_columns_same_dtype = all(df[col].apply(type).nunique() == 1 for col in df.columns)

if are_all_columns_same_dtype:
    print("Data type is the same for all values within each column.")
else:
    print("Data type varies within some columns.")

    # Identify columns where data type varies
    columns_with_different_dtype = [col for col in df.columns if df[col].apply(type).nunique() != 1]

    # Convert values to string for columns where data type varies
    for col in columns_with_different_dtype:
        df[col] = df[col].astype(str)

    print("Columns with varying data types have been converted to string.")


Data type varies within some columns.
Columns with varying data types have been converted to string.


In [22]:
import pandas as pd

# Specify the file path where you want to save the CSV file on your desktop
csv_file_path = 'C:\\Users\\User\\Desktop\\Tema Diplomes Master\\myfile.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

# Print a message to confirm that the file has been saved
print(f"CSV file saved at: {csv_file_path}")


CSV file saved at: C:\Users\User\Desktop\Tema Diplomes Master\myfile.csv


In [2]:
import mysql.connector
import pandas as pd
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load your dataset
logging.info("Loading dataset...")
df = pd.read_csv('autoscout24-germany-dataset.csv')
logging.info("Dataset loaded successfully.")

# Replace NaN values with None
logging.info("Replacing NaN values with None...")
df = df.where(pd.notnull(df), None)
logging.info("Replacement complete.")

# Database connection details
db_config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost'
}

# Create a new connection
logging.info("Connecting to the database...")
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Create database if it doesn't exist
logging.info("Creating database if it doesn't exist...")
create_database_query = "CREATE DATABASE IF NOT EXISTS `German Cars`"
cursor.execute(create_database_query)
logging.info("Database checked/created.")

# Use the new database
logging.info("Using the German Cars database...")
use_database_query = "USE `German Cars`"
cursor.execute(use_database_query)
logging.info("Database selected.")

# Create table if it doesn't exist
logging.info("Creating table if it doesn't exist...")
create_table_query = """
CREATE TABLE IF NOT EXISTS cars (
    id INT AUTO_INCREMENT PRIMARY KEY,
    mileage INT,
    make VARCHAR(50),
    model VARCHAR(50),
    fuel VARCHAR(50),
    gear VARCHAR(50),
    offerType VARCHAR(50),
    price DECIMAL(10, 2),
    hp INT,
    year INT
)
"""
cursor.execute(create_table_query)
logging.info("Table checked/created.")

# Insert the data from the DataFrame
insert_query = """
INSERT INTO cars (mileage, make, model, fuel, gear, offerType, price, hp, year)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

logging.info("Starting data insertion...")

# Adding a counter for logging progress
total_rows = len(df)
for index, row in enumerate(df.iterrows()):
    row_data = row[1].to_list()
    # Check for NaN values in the row_data
    row_data = [None if pd.isna(x) else x for x in row_data]
    try:
        cursor.execute(insert_query, tuple(row_data))
    except mysql.connector.Error as err:
        logging.error(f"Error inserting row {index}: {err}")
    if index % 10000 == 0:  # Log every 100 rows
        logging.info(f"Inserted {index}/{total_rows} rows.")

# Commit the transaction
logging.info("Committing the transaction...")
connection.commit()
logging.info("Transaction committed.")

# Close the connection
logging.info("Closing the database connection...")
cursor.close()
connection.close()
logging.info("Database connection closed.")

logging.info("Database, table, and data insertion completed successfully.")


2024-07-03 18:07:05,949 - INFO - Loading dataset...
2024-07-03 18:07:06,045 - INFO - Dataset loaded successfully.
2024-07-03 18:07:06,047 - INFO - Replacing NaN values with None...
2024-07-03 18:07:06,069 - INFO - Replacement complete.
2024-07-03 18:07:06,072 - INFO - Connecting to the database...
2024-07-03 18:07:06,085 - INFO - Creating database if it doesn't exist...
2024-07-03 18:07:06,095 - INFO - Database checked/created.
2024-07-03 18:07:06,098 - INFO - Using the German Cars database...
2024-07-03 18:07:06,100 - INFO - Database selected.
2024-07-03 18:07:06,101 - INFO - Creating table if it doesn't exist...
2024-07-03 18:07:06,113 - INFO - Table checked/created.
2024-07-03 18:07:06,115 - INFO - Starting data insertion...
2024-07-03 18:07:06,148 - INFO - Inserted 0/44265 rows.
2024-07-03 18:07:06,249 - INFO - Inserted 100/44265 rows.
2024-07-03 18:07:06,317 - INFO - Inserted 200/44265 rows.
2024-07-03 18:07:06,384 - INFO - Inserted 300/44265 rows.
2024-07-03 18:07:06,463 - INFO -